In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)
from sklearn.metrics import accuracy_score
from getData import getData
from knn import knn_masking_map, knn_masking_map_pure
from ot_dis.wtk import wtk_distance
from arabic_load import get_train_data, get_test_data

DEBUG:matplotlib:matplotlib data path: c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\Administrator\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\Administrator\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\Administrator\.matplotlib\fontlist-v330.json


# 1. Load data

In [3]:
X_train, y_train, X_test, y_test = getData('BME', '../data/UCR/raw_data/')
len(X_train[0]), len(X_test)

(128, 150)

In [5]:
X_train_china, y_train_china, X_test_china, y_test_china = getData('Chinatown', '../data/UCR/raw_data/')
len(X_train_china[0]), len(X_train)

(24, 30)

In [6]:
len(X_train), len(X_test), X_train[0].shape

(30, 150, (128,))

In [7]:
X_train_arabic, y_train_arabic = get_train_data()
X_test_arabic, y_test_arabic = get_test_data()
len(X_train_arabic), len(X_test_arabic)

(6600, 2200)

In [8]:
X_train_arabic = X_train_arabic[:660]
X_test_arabic = X_test_arabic[:100]
y_train_arabic = y_train_arabic[:660]
y_test_arabic = y_test_arabic[:100]

In [9]:
X_train_arabic[0].shape

(38, 13)

# 2. Training

## 2.1. Masking map

### 2.1.1. 1D data

In [8]:
lamb = 5
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="normal",lamb=lamb)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
  0%|          | 0/30 [00:01<?, ?it/s]


KeyboardInterrupt: 

### 2.1.2. 2D data

## 2.2. Partial masking map

### 2.2.1. 1D data

In [7]:
lamb = 5
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="partial",lamb=lamb, s=0.7)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [06:55<00:00, 13.86s/it]


0.9533333333333334

### 2.2.2. 2D data

## 2.4. Sequence masking map

In [8]:
len(X_train[0]), len(X_train_china[0])

(128, 24)

In [9]:
lamb = 5
sub_length = 25
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, sub_length=sub_length)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [04:19<00:00,  8.66s/it]


0.7266666666666667

In [10]:
sub_length = 25
lamb = 5
accuracy = knn_masking_map_pure(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, sub_length=sub_length)
accuracy

c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.7266666666666667

In [10]:
def knn_masking_map_pure(X_train, X_test, y_train, y_test, method="normal", lamb=5, s=0.5, sub_length=25, k=1):
    kpg_dict = {
        "sequence": masking_map_sequence,
        "normal": masking_map,
        "partial": masking_map_partial
    }
    clf = KNeighborsClassifier(n_neighbors = 1, metric=kpg_dict[method], metric_params={"lamb": lamb, "s": s, "sub_length": sub_length}, n_jobs=-1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    return accuracy

In [9]:
lamb = 5
sub_length = 40
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, s=0.7, sub_length=sub_length)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [03:15<00:00,  6.50s/it]


0.6466666666666666

In [11]:
sub_length = 40
lamb = 5
accuracy = knn_masking_map_pure(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, sub_length=sub_length)
accuracy

c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.6466666666666666

In [12]:
lamb = 5
sub_length = 120
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, s=0.7, sub_length=sub_length)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [00:23<00:00,  1.25it/s]


0.9066666666666666

In [15]:
lamb = 2
sub_length = 100
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, s=0.7, sub_length=sub_length)
accuracy

100%|██████████| 30/30 [00:34<00:00,  1.14s/it]


0.6666666666666666

In [21]:
lamb = 2
sub_length = 14
accuracy = knn_masking_map(X_train=X_train_china, X_test=X_test_china, y_train=y_train_china, y_test=y_test_china,  method="sequence",lamb=lamb, s=0.7, sub_length=sub_length)
accuracy

  0%|          | 0/20 [00:00<?, ?it/s]c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 20/20 [00:34<00:00,  1.75s/it]


0.9067055393586005

In [28]:
sub_length = len(X_train[0]) - 40
lamb = 2
accuracy = knn_masking_map(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="sequence",lamb=lamb, sub_length=sub_length)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [00:54<00:00,  1.82s/it]


0.5933333333333334

In [17]:
import numpy as np
def findTheShortestArray(list1, list2):
    min1 = min(list1, key=len)
    min2 = min(list2, key=len)
    min_len = min(len(min1), len(min2))
    frac1 = int(np.floor(min_len*10/100))
    frac2 = int(np.floor(min_len*30/100))
    frac3 = int(np.floor(min_len*50/100))
    frac4 = int(np.floor(min_len*70/100))
    frac5 = int(np.floor(min_len*90/100))
    min_array = [frac1, frac2, frac3, frac4, frac5]
    return min_array
size_list = [10, 30, 50, 70, 90]

In [18]:
X_train, y_train, X_test, y_test = getData('Chinatown', '../data/UCR/raw_data/')

In [19]:
size_array = findTheShortestArray(X_train, X_test)
size_array

[2, 7, 12, 16, 21]

In [20]:
for index, kernel in enumerate(size_array):
    with open('../experiments/experiment_MM_sequence_lambda_experiment.txt', 'a') as file:
        file.write(f"Chinatown({size_list[index]}) ")
    for i in range (1, 10):
        accuracy = knn_masking_map_pure(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, method="sequence", sub_length=kernel, k=1, lamb=i)
        with open('../experiments/experiment_MM_sequence_lambda_experiment.txt', 'a') as file:
            if i == 9:
                file.write(f"&{round(accuracy, 2)}\\\ \n")
            else:
                file.write(f"&{round(accuracy, 2)} ")

with open('../experiments/experiment_MM_sequence_lambda_experiment.txt', 'a') as file:
    file.write(f"--------------------------------------------------------------------------------------------------\n")

c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
c:\Users\Administr

In [6]:
for i in range(1, 10):
    accuracy = knn_masking_map_pure(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, method="sequence", sub_length=size_array[0], k=1, lamb=i)
    print(f"BME LAMBDA={i} accuracy={accuracy}")

c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


In [5]:
accuracy = knn_masking_map_pure(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, method="sequence", sub_length=size_array[0], k=1, lamb=1)
accuracy

c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


0.8333333333333334